In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


#### random forest

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_validate

X = pd.read_csv('final_training_data.csv', index_col=0).drop(columns=['grade', 'username'])
y = pd.read_csv('final_training_data.csv').loc[:,'grade'] #1 is failed grade, 0 is pass
print(y.value_counts())

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

scoring = ['average_precision', 'roc_auc', 'precision', 'recall', 'f1', 'accuracy']

rf = RandomForestClassifier()

# Best parameters: {'n_estimators': 1400, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 100, 'bootstrap': False}
#rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, n_jobs = -1, 
#                               scoring=scoring, refit='average_precision')
#rf_random.fit(X, y)
param_grid = {
    'bootstrap': [True],
    'max_depth': [90, 100, 110],
    'max_features': ['auto'],
    'min_samples_leaf': [3, 4, 5, 6],
    'min_samples_split': [3, 5, 6, 10],
    'n_estimators': [500, 1000, 1500,]
}
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 5, n_jobs = -1, verbose = 2, scoring=scoring, refit='average_precision')
grid_search.fit(X, y)

#scores = cross_validate(rf, X, y=y, scoring=scoring)
#print(f'scores: {scores}')

print("Best parameters: {}".format(grid_search.best_params_))
print(f'Best score: {grid_search.best_score_}')


0    183
1     37
Name: grade, dtype: int64
Fitting 5 folds for each of 144 candidates, totalling 720 fits
Best parameters: {'bootstrap': True, 'max_depth': 100, 'max_features': 'auto', 'min_samples_leaf': 6, 'min_samples_split': 5, 'n_estimators': 500}
Best score: 0.5376913887261097


In [12]:
pd.DataFrame(grid_search.cv_results_).to_csv('rf_results.csv')